조엘 그루스, 2016, 인사이트, '밑바닥부터 시작하는 데이터 과학'

In [1]:
from collections import defaultdict, Counter
from bs4 import BeautifulSoup
import requests

In [2]:
#데이터가져오기
def fix_unicode(text):
    return text.replace(u"\u2019","'")
from bs4 import BeautifulSoup
import requests
import re
url = "http://radar.oreilly.com/2010/06/what-is-data-science.html"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')
content = soup.find("div", "article-body")
regex = r"[\w']+|[\.]"

document = []


for paragraph in content("p"):
    words = re.findall(regex, fix_unicode(paragraph.text))
    document.extend(words)
print(document[:3])
print(len(document))

["We've", 'all', 'heard']
4990


### bi-gram

In [3]:
from collections import defaultdict
bigrams = zip(document, document[1:])
transitions = defaultdict(list)
for prev, current in bigrams:
    transitions[prev].append(current)

In [4]:
import random
def generate_using_bigrams():
    current = "."
    result = []
    while True:
        next_word_candidates = transitions[current]
        current = random.choice(next_word_candidates)
        result.append(current)
        if current ==".": return " ".join(result)

In [5]:
generate_using_bigrams()

'What differentiates data but how economies work with patience the discovery of about web has been instrumental in your searches correlates what those products are designed for and the missing points That joke that appears intractable .'

### tri-gram

In [6]:
trigrams = zip(document, document[1:], document[2:])
trigram_transitions = defaultdict(list)
starts = []

for prev, current, next in trigrams:
    if prev == ".":
        starts.append(current)
    trigram_transitions[(prev, current)].append(next)

In [7]:
def generate_using_trigrams():
    current = random.choice(starts)
    prev = "."
    result = [current]
    while True:
        next_word_candidates = trigram_transitions[(prev,current)]
        next_word = random.choice(next_word_candidates)
        
        prev, current = current, next_word
        result.append(current)
        
        if current == ".":
            return " ".join(result)

In [8]:
generate_using_trigrams()

'Traditional data analysis has been hampered by extremely long turn around times .'

쿠지라 히코우즈쿠,위키북스,파이썬을 이용한 머신러닝,딥러닝 실전 개발 입문, 312~314pp

In [9]:
# num은 N으로, 기준 단어에서 몇 앞의 문자를 볼지 계산
def ngram(s, num):
    res = []
    slen = len(s) - num + 1
    for i in range(slen):
        ss=s[i:i+num]
        res.append(ss)
    return res

def diff_ngram(sa, sb, num):
    a = ngram(sa, num)
    b = ngram(sb, num)
    r = []
    cnt = 0
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                r.append(i)
    return cnt / len(a), r

In [10]:
a = "오늘 강남에서 맛있는 스파게티를 먹었다."
b = "강남에서 먹었던 오늘의 스파게티는 맛있었다."

r2, word2 = diff_ngram(a, b, 2) # 2-gram
print("2-gram:", r2)
print(word2)
r3, word3 = diff_ngram(a, b, 3) # 3-gram
print("3-gram:", r3)
print(word3)

2-gram: 0.7619047619047619
['오늘', '강남', '남에', '에서', '서 ', ' 맛', '맛있', '는 ', ' 스', '스파', '파게', '게티', ' 먹', '먹었', '었다', '다.']
3-gram: 0.45
['강남에', '남에서', '에서 ', ' 맛있', ' 스파', '스파게', '파게티', ' 먹었', '었다.']


In [11]:

a = "머신러닝은 매우 재미있는 기술이라 공부하고 있습니다."
b = "공부하면 재미있는 기술이라 머신러닝을 배우고 있습니다."

r2, word2 = diff_ngram(a, b, 2) # 2-gram
print("2-gram:", r2)
print(word2)
r3, word3 = diff_ngram(a, b, 3) # 3-gram
print("3-gram:", r3)
print(word3)

2-gram: 0.75
['머신', '신러', '러닝', ' 재', '재미', '미있', '있는', '는 ', ' 기', '기술', '술이', '이라', '라 ', '공부', '부하', '고 ', ' 있', '있습', '습니', '니다', '다.']
3-gram: 0.6296296296296297
['머신러', '신러닝', ' 재미', '재미있', '미있는', '있는 ', '는 기', ' 기술', '기술이', '술이라', '이라 ', '공부하', '고 있', ' 있습', '있습니', '습니다', '니다.']
